In [1]:
#Here are some tests with Dask, which should enable parallel processing for increased speed.
#Leaving this as an example if someone else wants to try Dask
#Also testing NMF processing times
#Probably a lot to optimize here

#Works with python-data module. Geoconda seems to result in errors

In [2]:
from dask.distributed import Client

client = Client(n_workers=30, threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 30
Total threads: 30,Total memory: 76.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37837,Workers: 30
Dashboard: http://127.0.0.1:8787/status,Total threads: 30
Started: Just now,Total memory: 76.00 GiB
Comm: tcp://127.0.0.1:36407,Total threads: 1
Dashboard: http://127.0.0.1:32899/status,Memory: 2.53 GiB
Nanny: tcp://127.0.0.1:33041,


In [22]:
#import sys
#sys.path.append("../")
#from src import puhti_files

#TODO: Dask fails to read module from the above location. Maybe Client.upload_file()

In [ ]:
#Download needed data if needed
#Not sure with which logic the download folder is determined but probably best to include in gitignore
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

In [24]:
def read_text_file(document_id):
    # Construct the path to the .txt file based on the document_id
    file_path = '/scratch/project_2007227/genre_data/ecco_source/' + document_id + '.txt'

    # Read the contents of the .txt file into a string
    with open(file_path, 'r') as file:
        file_contents = file.read()
        
    return file_contents

In [12]:
def lower_text(text):
    text = text.lower()
    return text

In [13]:
import string

def remove_punctuation(text):
    # Remove special characters
    text = text.translate(str.maketrans('', '', string.punctuation)).replace("\n", " ")
    return text

In [14]:
from nltk import word_tokenize

def tokenize(text):
    # Tokenize words
    words = word_tokenize(text)
    return words

In [15]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

def remove_stop_words(words):
    # Remove English stop words
    
    words = [word for word in words if word.lower() not in stop_words]
    return words

In [16]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize(words):
    # Turn words to lemmas
    words = [lemmatizer.lemmatize(word) for word in words]
    return words

In [17]:
def join_words(words):
    text = " ".join(words)
    return text

In [18]:
import dask.bag as db
from dask_ml.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import HashingVectorizer
#Hashing is faster but doesn't provide information afterwards

vectorizer = CountVectorizer(max_features=10000)
#vectorizer = HashingVectorizer()

In [20]:
#Using dev set here

import dask.dataframe as dd

ddf = dd.read_csv(
    "/scratch/project_2007227/genre_data/dev.csv",
    sep=",",
    dtype={"document_id":str, "work_id":str, "main_category":int, "sub_category":int},
    blocksize="1KB"
)

In [25]:
%%time
meta = (None, "object")
ddf["text"] = (ddf.document_id.apply(read_text_file, meta=meta)
               .apply(lower_text, meta=meta)
               .apply(remove_punctuation, meta=meta)
               .apply(tokenize, meta=meta)
               .apply(remove_stop_words, meta=meta)
               .apply(lemmatize, meta=meta)
               .apply(join_words, meta=meta)
              )

result = ddf.compute(client=client)

CPU times: user 32.7 s, sys: 14.4 s, total: 47.2 s
Wall time: 6min 13s


In [26]:
%%time
#Lazy method, so doesn't compute without compute()
corpus = db.from_sequence(result["text"])

CPU times: user 8.18 s, sys: 870 ms, total: 9.05 s
Wall time: 8.83 s


In [27]:
%%time
X = vectorizer.fit_transform(corpus)

CPU times: user 15.6 s, sys: 4.28 s, total: 19.9 s
Wall time: 56.4 s


In [28]:
#Word list, but information is not really utilized here yet
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['00', '000', '0000', ..., 'óo', 'óris', 'ôter'], dtype=object)